In [1]:
%pip install langchain_community
%pip install azure-ai-ml
%pip install -qU langchain-openai
%pip install json5

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.llms.azureml_endpoint import (
    AzureMLOnlineEndpoint,
    AzureMLEndpointApiType,
    DollyContentFormatter,
    ContentFormatterBase
)
from typing import Dict
import urllib.request
import json
import os
import ssl
import time
from langchain_openai import AzureChatOpenAI
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter,
)
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint

import json5
import re
import time



#### 1st Agent: Fine-tuned LLM

In [ ]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

class CustomFormatter(ContentFormatterBase):
    content_type = "application/json"
    accepts = "application/json"

    def format_request_payload(self, prompt: str, *args, **kwargs) -> bytes:
        model_kwargs = kwargs.get('model_kwargs', {})
        input_str = json.dumps(
            {"input_data": [{"role": "user", "content": prompt}], "params": {"temperature": 0.1, "max_new_tokens": 1000, "do_sample": True}}
        )
        return str.encode(input_str)

    def format_response_payload(self, output: bytes, *args, **kwargs) -> Dict:
        response_json =  json.loads(output)

        try:
            # Step 1: Prepare JSON string
            response_content = response_json['result'].replace("'", "\"")

            # Step 2: Escape double quotes within the `content` fields
            response_content = re.sub(r'(?<=content": ")', lambda m: m.group(0).replace('"', '\\"'), response_content)
            response_content = response_content.replace('Here"s', 'Here\'s')
            
            # Step 3: Split the JSON string based on "} {" separator to handle multiple entries
            json_entries = response_content.split("} {")
            
            # Wrap entries in braces to create valid JSON for each entry
            json_entries = ["{" + entry + "}" if not entry.startswith("{") else entry for entry in json_entries]
            json_entries = [entry + "}" if not entry.endswith("}") else entry for entry in json_entries]
            
            # Step 4: Parse each JSON entry and retrieve assistant content
            assistant_content = None
            for json_string in json_entries:
                # Regular expression to find "role": "assistant" and capture "content" value
                match = re.search(r'"role"\s*:\s*"assistant",\s*"content"\s*:\s*"(.*?)"\s*}', json_string, re.DOTALL)

                # Check if role is "assistant" and retrieve content
                if match:
                    assistant_content = match.group(1)  # Get the content text
        
            return {"text": assistant_content}
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")

        # Return a default response if there was an error
            return {"text": "No valid response"}

# Example usage of the CustomFormatter
content_formatter = CustomFormatter()


solver_llm = AzureMLOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000, "do_sample": True},
    content_formatter=content_formatter,
)




#### Second Agent: RAG + Phi-3

In [ ]:
# Azure AI Search setup
AZURE_AI_SEARCH_SERVICE_NAME = ""
AZURE_AI_SEARCH_API_KEY = ""

retriever = AzureAISearchRetriever(
    service_name=AZURE_AI_SEARCH_SERVICE_NAME,
    index_name="vector-rag",
    api_key=AZURE_AI_SEARCH_API_KEY,
    content_key="chunk",
    top_k=1
)

# Azure OpenAI setup
coder_llm = AzureMLChatOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000},
)

#### Reasoning Agent: Phi-3

In [ ]:
reasoning_agent = AzureMLChatOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000},
)

#### Muti-Agent Logic

In [154]:
query_part1 = "Question 1: Samtools can be used to select reads above certain mapping quality. samtools view -h -b -q 30 aligned.bam -o above.mapQ30.bam But, how to select a read below certain mapping quality - all aligned reads below mapQ 30?</strong>I know it can be done using awk. But, the pipeline gets lengthy and time consuming when first need to convert bam to sam - separate header - use awk for mapQ below 30 - add header - sam file - convert to bam.Really, its taking a lots of time."
query_part2 = "Question 2: Provide your logic and reasoning."
combined_query = f"{query_part1}\n{query_part2}"
# Invoke the model and print the result
query = f"Question: {combined_query}.\nPlease provide a solution in raw text."

In [5]:
def format_docs(docs):
    """Format retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs if doc.page_content) if docs else ""


In [ ]:
def invoke_with_retries(llm, prompt, max_retries=5, backoff_factor=2):
    """
    Helper function to invoke LLM with retry logic in case of timeout or HTTP errors.
    """
    for attempt in range(max_retries):
        try:
            return llm.invoke(prompt, timeout=300000)  # Attempt to invoke LLM
        except TimeoutError:
            # Handle timeout errors with exponential backoff
            sleep_time = backoff_factor * (2 ** attempt)
            time.sleep(sleep_time)
            print(f"Timeout occurred. Retrying after {sleep_time} seconds... Attempt {attempt + 1}/{max_retries}")
        except HTTPError as e:
            if e.code == 424:  # Handle HTTP 424: Failed Dependency
                print(f"HTTP Error 424: Failed Dependency. Skipping after {attempt + 1}/{max_retries} attempts.")
                return None
            else:
                print(f"HTTP Error {e.code}: {e.reason}. Retrying...")
                sleep_time = backoff_factor * (2 ** attempt)
                time.sleep(sleep_time)
    print(f"Max retries reached for {llm}.")
    return None  # Return None if all retries fail

In [121]:
def generate_final_response(query, retries=3, backoff_factor=1):
    for attempt in range(retries):

        # Step 1: Retrieve documents
        docs = retriever.invoke(query)
        if not docs:
            print("No documents found for the query.")

        # Step 2: Format documents
        context = format_docs(docs)
        if not context:
            print("Formatted context is empty.")

        # Step 3: Generate prompt for coder LLM
        prompt = f"This is relevant code documentation: \n{context}\n\nQuestion: {query}.\nPlease respond with code."
        response_coder = coder_llm.invoke(prompt)

        # Process coder response
        response_code = response_coder.content if hasattr(response_coder, 'content') else ""
        if not response_code:
            print("Unexpected response from coder LLM.")

        # Step 4: Generate solver response with retry on timeout
        solver_text = invoke_with_retries(solver_llm, prompt, max_retries=6)
        if solver_text is None:
            print("Failed to get response from solver LLM after retries.")
            solver_text = ""

        # Step 5: Generate reasoning prompt
        prompt_reason = (
            f"You are an AI reasoning agent collaborating with other specialized assistants. "
            f"There are two previous responses you should review: "
            f"A solving agent specializing in bioinformatics tools has provided the following response: {solver_text}. "
            f"A coder agent specializing in genomics analysis workflows written in Nextflow has provided the following response: {response_code}. "
            f"Your task is to answer the user query: {query}. "
            f"After you generate your response, print the quality rating of your output as Quality Rating: on a new line. using the scale 1-10 using format x/10."
        )

        combined_reasoning = reasoning_agent.invoke(prompt_reason)

        # Step 6: Check reasoning response
        if hasattr(combined_reasoning, 'content'):
            final_response = combined_reasoning.content

            match = re.search(r'Quality Rating: (\d+)/10', final_response)

            if match:
                quality_rating = int(match.group(1))
                if quality_rating > 7:
                    print("Achieved satisfactory rating.")
                    return final_response  # Return response if rating is above 7
                else:
                    print("Rating not satisfactory, continuing to next attempt.")
            else:
                print("Quality Rating not found in the response.")
        else:
            print("Reasoning agent did not return valid content.")

        # Backoff before the next attempt
        sleep_time = backoff_factor * (2 ** attempt)  # Exponential backoff
        print(f"Sleeping for {sleep_time} seconds before next attempt.")
        time.sleep(sleep_time)

    print("Max attempts reached. Returning last response.")
    return final_response if 'final_response' in locals() else None


In [180]:
query_part1= ["Student ask: 'How would I provide quality metric on fastq files?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to provide quality metric on fastq files?' Provide the steps you would advise the student to take.",
              "Student ask: 'How do I align RNA-seq data against human reference genome?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to align RNA-seq data against human reference genome?' Provide the steps you would advise the student to take.",
              "Student ask: 'How can I assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' Provide the steps you would advise the student to take."
              ]

query_part2= "Please explain your logic and reasoning behind your answer."
query_part3 = "What additional information do you need to answer the question?"

results = {"easy_q" : "", "easy_code" : "", "med_q": "" , "med_code": "", "hard_q": "", "hard_code": ""}

for idx, question in enumerate (query_part1):

  combined_query = f"{question}\n{query_part2}\n{query_part3}"
  query = f"Question: {combined_query}\nPlease provide a solution in raw text."
  print (query)
  final_response = generate_final_response(query)

  if idx == 0:
    results["easy_q"] = final_response

  elif idx == 1:
    results["easy_code"] = final_response

  elif idx == 2:
    results["med_q"] = final_response

  elif idx == 3:
    results["med_code"] = final_response

  elif idx == 4:
    results["hard_q"] = final_response

  elif idx == 5:
    results["hard_code"] = final_response



Question: Student ask: 'How would I provide quality metric on fastq files?' Provide the steps you would advise the student to take.
Please explain your logic and reasoning behind your answer.
What additional information do you need to answer the question?
Please provide a solution in raw text.
Achieved satisfactory rating.
Question: Student ask: 'What code or workflow do I need to write to provide quality metric on fastq files?' Provide the steps you would advise the student to take.
Please explain your logic and reasoning behind your answer.
What additional information do you need to answer the question?
Please provide a solution in raw text.
Trying Solver again
Trying Solver again
Trying Solver again
Max retries reached for AzureMLOnlineEndpoint
Params: {'deployment_name': '', 'model_kwargs': {'temperature': 0.1, 'max_new_tokens': 100, 'do_sample': True}}.
Failed to get response from solver LLM after retries.
Achieved satisfactory rating.
Question: Student ask: 'How do I align RNA-se

In [126]:
query_part1= [
              "Student ask: 'What code or workflow do I need to write to assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' "
              ]

query_part2= "Please explain your logic and reasoning behind your answer."
query_part3 = "What additional information do you need to answer the question?"



for idx, question in enumerate (query_part1):

  combined_query = f"{question}\n{query_part2}\n{query_part3}"
  query = f"Question: {combined_query}\nPlease provide a solution in raw text."
  print (query)
  final_response = generate_final_response(query)
  print(final_response)




Question: Student ask: 'What code or workflow do I need to write to assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' 
Please explain your logic and reasoning behind your answer.
What additional information do you need to answer the question?
Please provide a solution in raw text.
Achieved satisfactory rating.
Response:

To assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus, you can follow the workflow provided by the coder agent specializing in genomics analysis workflows written in Nextflow. This workflow uses tools like BWA, SAMtools, GATK, and custom scripts.

Here's an example of a workflow using command-line tools:

1. Quality control and trimming:
   - Use tools like FastQC and Trimmomatic to assess the quality of raw sequencing reads and trim low-quality bases and adapters.

2. Genome assembly:
   - Use a de novo 

In [198]:
# Open a file to save the output
with open("multiagent_output.txt", "w") as file:
    for key, value in results.items():
        # Print each key-value pair
        print(f"{key}: {value}")
        
        # Write each key-value pair to the file
        file.write(f"{key}: {value}\n")

easy_q: Quality Rating: 9/10

To provide quality metrics on fastq files, I would advise the student to follow these steps:

1. Choose a quality metric: The student should decide on the quality metric they want to use, such as Phred score, Q-score, or GC content. The Phred score is a popular choice as it represents the probability of an incorrect base call on a logarithmic scale.

2. Install necessary software: The student should install a tool that can calculate the chosen quality metric for fastq files. FastQC is a widely used quality control tool for high-throughput sequence data.

3. Run FastQC on the fastq files: The student should use the FastQC command-line tool to generate a quality report for their fastq files. The command would look like this:

```bash
fastqc *.fastq.gz
```

This command will generate a report in HTML format, which the student can open in a web browser to view the quality metrics.

4. Analyze the FastQC report: The FastQC report will contain several quality me